<a href="https://colab.research.google.com/github/juanjo1228/Proyecto-IA/blob/main/99-modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================================================
# IMPORTACIONES OBLIGATORIAS (EJECUTAR PRIMERO SIEMPRE)
# ===============================================================

import pandas as pd
import numpy as np
from google.colab import files
from zipfile import ZipFile

# Preprocesamiento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modelo
from xgboost import XGBClassifier


In [ ]:
# ===============================================================
#      SUBIR ARCHIVO ZIP QUE CONTIENE train.csv Y test.csv
# ===============================================================
print("Sube el archivo ZIP que contiene train.csv y test.csv...")
uploaded = files.upload()

# Obtener nombre del ZIP subido
zip_filename = list(uploaded.keys())[0]

# Extraer ZIP
print("\nDescomprimiendo ZIP...")
with zipfile.ZipFile(zip_filename, 'r') as z:
    z.extractall("dataset")

print("Archivos extraídos:")
print(os.listdir("dataset"))

# Buscar train.csv y test.csv automáticamente
train_path = None
test_path = None

for root, dirs, files_in_dir in os.walk("dataset"):
    for f in files_in_dir:
        if f.lower() == "train.csv":
            train_path = os.path.join(root, f)
        if f.lower() == "test.csv":
            test_path = os.path.join(root, f)

if train_path is None or test_path is None:
    raise Exception("No se encontró train.csv o test.csv dentro del ZIP.")

print("\nUbicación detectada:")
print("Train:", train_path)
print("Test :", test_path)

# ===============================================================
#                 CARGA DE ARCHIVOS
# ===============================================================
train_df = pd.read_csv(train_path)
test_df  = pd.read_csv(test_path)


Sube el archivo ZIP que contiene train.csv y test.csv...


Saving udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip to udea-ai-4-eng-20252-pruebas-saber-pro-colombia (3).zip

Descomprimiendo ZIP...
Archivos extraídos:
['test.csv', 'submission_example.csv', 'train.csv']

Ubicación detectada:
Train: dataset/train.csv
Test : dataset/test.csv


In [ ]:
train_df.columns


Index(['ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2',
       'INDICADOR_3', 'INDICADOR_4'],
      dtype='object')

In [ ]:

#                 TARGET
# ===============================================================
target = "RENDIMIENTO_GLOBAL"

if target not in train_df.columns:
    raise Exception(f"La variable objetivo '{target}' NO está en train.csv")

X = train_df.drop(columns=[target])
y = train_df[target]

# ===============================================================
#     MAPEO DE CLASES (OBLIGATORIO PARA XGBOOST)
# ===============================================================
label_map = {
    "bajo": 0,
    "medio-bajo": 1,
    "medio-alto": 2,
    "alto": 3
}
y = y.map(label_map)

# ===============================================================
#     COLUMNAS CATEGÓRICAS Y NUMÉRICAS
# ===============================================================
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

print("\nColumnas categóricas detectadas:", list(cat_cols))
print("Columnas numéricas detectadas:", list(num_cols))

# ===============================================================
#     PREPROCESAMIENTO CON ONE-HOT ENCODING
# ===============================================================
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ]
)

# ===============================================================
#     DEFINICIÓN DEL MODELO XGBOOST
# ===============================================================
model = XGBClassifier(
    n_estimators=400,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="multi:softmax",
    num_class=4,
    tree_method="hist",
    random_state=42
)

# Pipeline
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
])

# ===============================================================
#     TRAIN / VALIDACIÓN
# ===============================================================
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# ===============================================================
#     ENTRENAMIENTO
# ===============================================================
print("\nEntrenando modelo...")
clf.fit(X_train, y_train)

# ===============================================================
#     EVALUACIÓN
# ===============================================================
val_acc = clf.score(X_val, y_val)
print(f"\nAccuracy en validación: {val_acc:.4f}")

# ===============================================================
#     PREDICCIONES SOBRE TEST
# ===============================================================
y_pred = clf.predict(test_df)

# Convertir números → palabras
inverse_label_map = {v: k for k, v in label_map.items()}
y_pred_labels = [inverse_label_map[i] for i in y_pred]

# ===============================================================
#     CREAR ARCHIVO DE SALIDA
# ===============================================================
submission = pd.DataFrame({
    "ID": test_df["ID"],
    "RENDIMIENTO_GLOBAL": y_pred_labels
})

submission.to_csv("out.csv", index=False)
print("\nArchivo out.csv generado correctamente.")

files.download("out.csv")


Columnas categóricas detectadas: ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE']
Columnas numéricas detectadas: ['ID', 'PERIODO_ACADEMICO', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

Entrenando modelo...

Accuracy en validación: 0.4304

Archivo out.csv generado correctamente.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pd.read_csv("out.csv")


,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo
...,...,...
296781,496981,medio-bajo
296782,209415,alto
296783,239074,medio-alto
296784,963852,alto
